In [1]:
import csv
import json
import mysql.connector

with open('hw5_config.json') as f:
    config = json.load(f)

host = config['host']
user = config['user']
passwd = config['passwd']
dbms_student_path = config['dbms_student_path']
new_student_path = config['new_student_path']

## 1-1: Input data into student table

In [2]:
import pandas as pd

mydb = mysql.connector.connect(host = host, user = user, passwd = passwd)
mycursor = mydb.cursor()
mycursor2 = mydb.cursor(prepared = True)

mycursor.execute("CREATE DATABASE IF NOT EXISTS DB_class")
mydb.commit()

mycursor.execute("USE DB_class")

mycursor.execute("CREATE TABLE IF NOT EXISTS student (身份 varchar(255), 系所 varchar(255), 年級 int, 學號 varchar(255) PRIMARY KEY, 姓名 varchar(255), 信箱 varchar(255), 班別 varchar(255))")
mydb.commit()

df = pd.read_csv(dbms_student_path, encoding = 'big5')
df = df.where(pd.notnull(df), None)

def trim(x):
    x = x.strip() if isinstance(x, str) else x
    return x

for index, row in df.iterrows():
    row = [trim(cell) for cell in row]
    mycursor2.execute("INSERT INTO student (身份, 系所, 年級, 學號, 姓名, 信箱, 班別) VALUES (%s, %s, %s, %s, %s, %s, %s)", tuple(row))
    mydb.commit()

## 1-2: Read the information about yourself

In [3]:
mycursor.execute("SELECT * FROM student WHERE 學號 = 'R12922116'")
print(mycursor.fetchall()[0])

('學生', '資工所', 1, 'R12922116', '王偉力 (WANG, WEI-LI)', 'r12922116@ntu.edu.tw', '資料庫系統-從SQL到NoSQL (EE5178)')


## 1-3: Update your own value of the "身份" column 

In [4]:
mycursor.execute("UPDATE student SET 身份 = '特優生' WHERE 學號 = 'R12922116'")
mydb.commit()

mycursor.execute("SELECT * FROM student WHERE 學號 = 'R12922116'")
print(mycursor.fetchall()[0])

('特優生', '資工所', 1, 'R12922116', '王偉力 (WANG, WEI-LI)', 'r12922116@ntu.edu.tw', '資料庫系統-從SQL到NoSQL (EE5178)')


## 1-4: Insert three new students 

In [5]:
df4 = pd.read_csv(new_student_path, encoding = 'utf-8')
df4 = df4.where(pd.notnull(df4), None)

for index, row in df4.iterrows():
    row = [trim(cell) for cell in row]
    mycursor2.execute("INSERT INTO student (身份, 系所, 年級, 學號, 姓名, 信箱, 班別) VALUES (%s, %s, %s, %s, %s, %s, %s)", tuple(row))
    mydb.commit()

## 1-5: Prepared select statement 

In [6]:
pre_stmt = "SELECT * FROM student WHERE 學號 = %s"
student_ids = ['R10123456', 'B09987653', 'R11123001']

for student_id in student_ids:
    mycursor2.execute(pre_stmt, (student_id,))
    print(mycursor2.fetchall()[0])

('旁聽生', '電機所', 2, 'R10123456', '小紅', 'r10123456@ntu.edu.tw', '資料庫系統-從SQL到NoSQL (EE5178)')
('學生', '物理系', 3, 'B09987653', '小黃', 'b09987653@ntu.edu.tw', '資料庫系統-從SQL到NoSQL (EE5178)')
('觀察者', '電信所', 1, 'R11123001', '小綠', 'r11123001@ntu.edu.tw', '資料庫系統-從SQL到NoSQL (EE5178)')


## 1-6: Group information

In [7]:
#(1)
mycursor.execute("ALTER TABLE student ADD COLUMN `group` int DEFAULT 0, ADD COLUMN `captain` int DEFAULT 0")
mydb.commit()

#(2)
mycursor.execute("UPDATE student SET `group` = 7 WHERE 學號 IN ('R12921059', 'R12922116', 'R12921093', 'B09502132')")
mydb.commit()

#(3)
mycursor.execute("UPDATE student SET `captain` = 1 WHERE 學號 = 'R12921059'")
mydb.commit()

#(4)
mycursor.execute("SELECT * FROM student WHERE `group` = 7")
for row in mycursor.fetchall():
    print(row)
    
mycursor.close()
mycursor2.close()
mydb.close()

('學生', '機械系', 4, 'B09502132', '周哲瑋 (CHOU,CHE-WEI)', 'b09502132@ntu.edu.tw', '資料庫系統-從SQL到NoSQL (EE5178)', 7, 0)
('學生', '電機所', 1, 'R12921059', '鄧雅文 (TENG, YA-WEN)', 'r12921059@ntu.edu.tw', '資料庫系統-從SQL到NoSQL (EE5178)', 7, 1)
('學生', '電機所', 1, 'R12921093', '吳吉加 (CHI-CHIA WU)', 'r12921093@ntu.edu.tw', '資料庫系統-從SQL到NoSQL (EE5178)', 7, 0)
('特優生', '資工所', 1, 'R12922116', '王偉力 (WANG, WEI-LI)', 'r12922116@ntu.edu.tw', '資料庫系統-從SQL到NoSQL (EE5178)', 7, 0)
